In [1]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib as plt
import scipy

In [2]:
# connect to sql
connection = psycopg2.connect(
    host = 'diasterdb2.cbyvxmyzmzyh.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'group7_db',
    database='diasterdb'
    )
cursor=connection.cursor()

In [3]:
# list tables in sql
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

,table_name,column_name,data_type,table_schema
0,FPIS,NAME,character varying,public
1,FPIS,FIPS_ID,integer,public
2,TEMPERATURE,DATE,date,public
3,TEMPERATURE,Country,character varying,public
4,TEMPERATURE,AverageTemperatureUncertaintyF,numeric,public
5,TEMPERATURE,STATE,character varying,public
6,TEMPERATURE,TEMPERATURE_ID,integer,public
7,TEMPERATURE,Fahrenheit,numeric,public
8,TEMPERATURE,AverageTemperatureF,numeric,public
9,USDisaster,pa_program_declared,integer,public


In [4]:
# dictionary to convert between full state names and abbreviations
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [5]:
#store sql table in dataframe
sql_USDisaster = """ SELECT * FROM public."USDisaster" """
df_US_disasters = pd.read_sql(sql_USDisaster, con=connection)
df_US_disasters.tail(3)

,fema_declaration,disaster_number,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,ih_program_declared,ia_program_declared,pa_program_declared,hm_program_declared,incident_begin_date,fips,place_code,designated_area,declaration_request_number,hash,last_refresh,id
61803,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6035,99035,Lassen (County),21087,74f77c93f158659bf1dd9f871227a9993244478d,2021-08-25,6125a937434e34d5ee5cd8a3
61804,DR-4610-CA,4610,CA,DR,2021-08-24,2021,Fire,Wildfires,1,0,1,1,2021-07-14,6057,99057,Nevada (County),21087,dde89f65556f220034ac16cfa66abf7d135962da,2021-08-25,6125a937434e34d5ee5cd8c1
61805,FM-5416-CA,5416,CA,FM,2021-08-25,2021,Fire,French Fire,0,0,1,1,2021-08-24,6029,99029,Kern (County),21088,b2120fdada11e2ceefe3714927afe2443c719812,2021-08-25,61266e5e434e34d5eee6f25e


In [6]:
# extract out month and year and make separate columns -- use this later to group by month/year
df_US_disasters['declaration_date'].astype(str)
df_US_disasters['year'] = pd.DatetimeIndex(df_US_disasters['declaration_date']).year
df_US_disasters['month'] = pd.DatetimeIndex(df_US_disasters['declaration_date']).month
df_US_disasters['month_year'] = pd.to_datetime(df_US_disasters['declaration_date']).dt.to_period('M')

In [7]:
df_US_disasters=df_US_disasters.sort_values(by = "state", ascending= True)
df_US_disasters=df_US_disasters.rename(columns = {"state":"STATE"})

df_US_disasters.head()

,fema_declaration,disaster_number,STATE,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,ih_program_declared,ia_program_declared,...,fips,place_code,designated_area,declaration_request_number,hash,last_refresh,id,year,month,month_year
35447,DR-1796-AK,1796,AK,DR,2008-09-26,2008,Severe Storm(s),"Severe Storms, Flooding, Landslides, And Mudsl...",0,0,...,2090,99090,Fairbanks North Star (Borough),8159,8d89c28a38fbe78b2b96c4320cd60371b9821b24,2021-07-13,60c3b848a0ee349d71090f10,2008,9,2008-09
51103,EM-3446-AK,3446,AK,EM,2020-03-13,2020,Biological,Covid-19,0,0,...,2261,14430,Chugach Regional Educational Attendance Area,20019,d8f4a89f3e5446bb519717956c431b68d6a73718,2021-07-13,60c3b85ea0ee349d710a977f,2020,3,2020-03
51104,EM-3446-AK,3446,AK,EM,2020-03-13,2020,Biological,Covid-19,0,0,...,2270,37840,Kashunamiut Regional Educational Attendance Area,20019,a795685e0418a3cf1b9d3c4fdfadd1167785b21f,2021-07-13,60c3b85ea0ee349d710a9781,2020,3,2020-03
22721,DR-1440-AK,1440,AK,DR,2002-11-08,2003,Earthquake,Earthquake,0,0,...,2170,99170,Matanuska-Susitna (Borough),2145,009b6e8a5b894824c8197c202c9a3a1643fb3941,2021-07-13,60c3b841a0ee349d71088040,2002,11,2002-11
22722,DR-1440-AK,1440,AK,DR,2002-11-08,2003,Earthquake,Earthquake,0,0,...,2240,99240,Southeast Fairbanks (Census Area),2145,1ba389d0b1edc6f1965ea3d77a88615ba492ee96,2021-07-13,60c3b841a0ee349d71088046,2002,11,2002-11


In [8]:
df_US_disasters.to_csv('US_Disasters_MY', index=False)

In [9]:
df_disaster = pd.read_csv('US_disasters_MY')

In [10]:
# get temperature table from sql
sql_temp = """ SELECT * FROM public."TEMPERATURE" """
df_US_temperatures = pd.read_sql(sql_temp, con=connection)

In [11]:
#same idea as above with disasters table; extract out month and year from date and add into separate columns
df_US_temperatures['DATE'].astype(str)
df_US_temperatures['year'] = pd.DatetimeIndex(df_US_temperatures['DATE']).year
df_US_temperatures['month'] = pd.DatetimeIndex(df_US_temperatures['DATE']).month
df_US_temperatures['month_year'] = pd.to_datetime(df_US_temperatures['DATE']).dt.to_period('M').astype(str)
df_US_temperatures.head()

,TEMPERATURE_ID,DATE,STATE,Country,Fahrenheit,AverageTemperatureF,AverageTemperatureUncertaintyF,year,month,month_year
0,1,1953-01-01,Alabama,United States,119.381,49.4762,32.2808,1953,1,1953-01
1,2,1953-02-01,Alabama,United States,118.634,49.3268,32.2286,1953,2,1953-02
2,3,1953-03-01,Alabama,United States,164.363,58.4726,32.1728,1953,3,1953-03
3,4,1953-04-01,Alabama,United States,175.298,60.6596,32.2916,1953,4,1953-04
4,5,1953-05-01,Alabama,United States,241.619,73.9238,32.3420,1953,5,1953-05


In [12]:
#convert to state abbreviations
df_US_temperatures['STATE'] = df_US_temperatures['STATE'].map(us_state_abbrev).fillna(df_US_temperatures['STATE'])
df_US_temperatures.sort_values(by = "STATE", ascending= True)


,TEMPERATURE_ID,DATE,STATE,Country,Fahrenheit,AverageTemperatureF,AverageTemperatureUncertaintyF,year,month,month_year
1092,1093,1983-04-01,AK,United States,-6.106,24.3788,32.7308,1983,4,1983-04
976,977,1973-08-01,AK,United States,110.750,47.7500,32.6120,1973,8,1973-08
975,976,1973-07-01,AK,United States,130.658,51.7316,32.5994,1973,7,1973-07
974,975,1973-06-01,AK,United States,111.965,47.9930,32.4140,1973,6,1973-06
973,974,1973-05-01,AK,United States,57.812,37.1624,32.7182,1973,5,1973-05
...,...,...,...,...,...,...,...,...,...,...
36693,36694,1973-06-01,WY,United States,166.622,58.9244,32.2682,1973,6,1973-06
36694,36695,1973-07-01,WY,United States,194.396,64.4792,32.4068,1973,7,1973-07
36695,36696,1973-08-01,WY,United States,198.842,65.3684,32.5346,1973,8,1973-08
36697,36698,1973-10-01,WY,United States,97.511,45.1022,32.4608,1973,10,1973-10


In [13]:
df_US_temperatures.to_csv('US_temperatures_MY', index = False)

In [14]:
#read in edited dfs
df_temp = pd.read_csv('US_temperatures_MY')
df_disaster = pd.read_csv('US_disasters_MY')

In [15]:
#df_temp = df_US_temperatures
#df_disaster = df_US_disasters

In [16]:
df_temp=df_US_temperatures.sort_values(by = "STATE", ascending= True)
df_temp.head(5)

,TEMPERATURE_ID,DATE,STATE,Country,Fahrenheit,AverageTemperatureF,AverageTemperatureUncertaintyF,year,month,month_year
1092,1093,1983-04-01,AK,United States,-6.106,24.3788,32.7308,1983,4,1983-04
976,977,1973-08-01,AK,United States,110.750,47.7500,32.6120,1973,8,1973-08
975,976,1973-07-01,AK,United States,130.658,51.7316,32.5994,1973,7,1973-07
974,975,1973-06-01,AK,United States,111.965,47.9930,32.4140,1973,6,1973-06
973,974,1973-05-01,AK,United States,57.812,37.1624,32.7182,1973,5,1973-05


In [17]:
df_disaster.drop(columns = ['disaster_number', 'fema_declaration', 'ih_program_declared', 'ia_program_declared'])

,STATE,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,pa_program_declared,hm_program_declared,incident_begin_date,fips,place_code,designated_area,declaration_request_number,hash,last_refresh,id,year,month,month_year
0,AK,DR,2008-09-26,2008,Severe Storm(s),"Severe Storms, Flooding, Landslides, And Mudsl...",1,1,2008-07-27,2090,99090,Fairbanks North Star (Borough),8159,8d89c28a38fbe78b2b96c4320cd60371b9821b24,2021-07-13,60c3b848a0ee349d71090f10,2008,9,2008-09
1,AK,EM,2020-03-13,2020,Biological,Covid-19,1,0,2020-01-20,2261,14430,Chugach Regional Educational Attendance Area,20019,d8f4a89f3e5446bb519717956c431b68d6a73718,2021-07-13,60c3b85ea0ee349d710a977f,2020,3,2020-03
2,AK,EM,2020-03-13,2020,Biological,Covid-19,1,0,2020-01-20,2270,37840,Kashunamiut Regional Educational Attendance Area,20019,a795685e0418a3cf1b9d3c4fdfadd1167785b21f,2021-07-13,60c3b85ea0ee349d710a9781,2020,3,2020-03
3,AK,DR,2002-11-08,2003,Earthquake,Earthquake,1,1,2002-11-03,2170,99170,Matanuska-Susitna (Borough),2145,009b6e8a5b894824c8197c202c9a3a1643fb3941,2021-07-13,60c3b841a0ee349d71088040,2002,11,2002-11
4,AK,DR,2002-11-08,2003,Earthquake,Earthquake,1,1,2002-11-03,2240,99240,Southeast Fairbanks (Census Area),2145,1ba389d0b1edc6f1965ea3d77a88615ba492ee96,2021-07-13,60c3b841a0ee349d71088046,2002,11,2002-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61801,WY,DR,2011-07-22,2011,Severe Storm(s),"Severe Storms, Flooding, And Landslides",1,1,2011-05-18,56000,84200,Wind River Indian Reservation,11166,888ad26eb9ed64bf78e461c67db7a3d44e834127,2021-07-13,60c3b862a0ee349d710af237,2011,7,2011-07
61802,WY,DR,2011-07-22,2011,Severe Storm(s),"Severe Storms, Flooding, And Landslides",1,1,2011-05-18,56001,99001,Albany (County),11166,5270a762b6be8a7f245cbbc8569c93618522db4e,2021-07-13,60c3b862a0ee349d710af239,2011,7,2011-07
61803,WY,DR,2020-04-11,2020,Biological,Covid-19 Pandemic,1,1,2020-01-20,56025,99025,Natrona (County),20153,52865666e0c845b48f965ce9bd75c27c501bda86,2021-08-06,60c3b873a0ee349d710bd520,2020,4,2020-04
61804,WY,EM,2020-03-13,2020,Biological,Covid-19,1,0,2020-01-20,56017,99017,Hot Springs (County),20066,0a119d4b83190f9f7f07f7ab916fe1ed3c2e771a,2021-07-13,60c3b860a0ee349d710ac6a2,2020,3,2020-03


In [18]:
df_c = pd.merge(df_disaster, df_temp,  how='inner', on=['month_year', 'STATE'])

In [19]:
df_c.columns



Index(['fema_declaration', 'disaster_number', 'STATE', 'declaration_type',
       'declaration_date', 'fy_declared', 'incident_type', 'declaration_title',
       'ih_program_declared', 'ia_program_declared', 'pa_program_declared',
       'hm_program_declared', 'incident_begin_date', 'fips', 'place_code',
       'designated_area', 'declaration_request_number', 'hash', 'last_refresh',
       'id', 'year_x', 'month_x', 'month_year', 'TEMPERATURE_ID', 'DATE',
       'Country', 'Fahrenheit', 'AverageTemperatureF',
       'AverageTemperatureUncertaintyF', 'year_y', 'month_y'],
      dtype='object')

In [20]:
df_c.isnull().sum()

fema_declaration                  0
disaster_number                   0
STATE                             0
declaration_type                  0
declaration_date                  0
fy_declared                       0
incident_type                     0
declaration_title                 0
ih_program_declared               0
ia_program_declared               0
pa_program_declared               0
hm_program_declared               0
incident_begin_date               0
fips                              0
place_code                        0
designated_area                   0
declaration_request_number        0
hash                              0
last_refresh                      0
id                                0
year_x                            0
month_x                           0
month_year                        0
TEMPERATURE_ID                    0
DATE                              0
Country                           0
Fahrenheit                        0
AverageTemperatureF         

In [21]:
len(df_c["disaster_number"])

40732

In [22]:
df_c = df_c.drop(columns = ['fema_declaration', 'ih_program_declared', 'ia_program_declared', 'pa_program_declared',
       'hm_program_declared','incident_begin_date'])


In [23]:
#drop biological
df_c = df_c[df_c['incident_type']!= 'Biological']

df_c.to_csv('combined_dataframe_State_date')

In [24]:
df_c.groupby('DATE')['incident_type'].count()

DATE
1953-05-01      2
1953-06-01      6
1953-07-01      1
1953-10-01      1
1953-12-01      1
             ... 
2013-05-01    125
2013-06-01     51
2013-07-01    171
2013-08-01     45
2013-09-01     84
Name: incident_type, Length: 589, dtype: int64

In [25]:
df_c.drop(columns = ['DATE', 'year_x', 'month_x', 
         'fy_declared', 'month_y', 'year_y','hash', 'last_refresh', 'id', "declaration_type", "AverageTemperatureUncertaintyF"])

,disaster_number,STATE,declaration_date,incident_type,declaration_title,fips,place_code,designated_area,declaration_request_number,month_year,TEMPERATURE_ID,Country,Fahrenheit,AverageTemperatureF
0,1796,AK,2008-09-26,Severe Storm(s),"Severe Storms, Flooding, Landslides, And Mudsl...",2090,99090,Fairbanks North Star (Borough),8159,2008-09,1398,United States,74.696,40.5392
1,1796,AK,2008-09-26,Severe Storm(s),"Severe Storms, Flooding, Landslides, And Mudsl...",2068,99068,Denali (Borough),8159,2008-09,1398,United States,74.696,40.5392
2,1796,AK,2008-09-26,Severe Storm(s),"Severe Storms, Flooding, Landslides, And Mudsl...",2185,99185,North Slope (Borough),8159,2008-09,1398,United States,74.696,40.5392
3,1796,AK,2008-09-26,Severe Storm(s),"Severe Storms, Flooding, Landslides, And Mudsl...",2290,86650,Yukon Koyukuk Regional Educational Attendance ...,8159,2008-09,1398,United States,74.696,40.5392
4,1440,AK,2002-11-08,Earthquake,Earthquake,2170,99170,Matanuska-Susitna (Borough),2145,2002-11,1328,United States,-27.373,20.1254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40727,2460,WY,2002-08-31,Fire,Wy - Commissary Ridge Fire - 08/31/02,56023,99023,Lincoln (County),2126,2002-08,37044,United States,187.196,63.0392
40728,1268,WY,1999-02-17,Severe Ice Storm,Wy - Winter Storm 10/5/98,56027,99027,Niobrara (County),99017,1999-02,37002,United States,15.089,28.6178
40729,1268,WY,1999-02-17,Severe Ice Storm,Wy - Winter Storm 10/5/98,56015,99015,Goshen (County),99017,1999-02,37002,United States,15.089,28.6178
40730,2993,WY,2012-07-02,Fire,Squirrel Creek Fire,56001,99001,Albany (County),12050,2012-07,37163,United States,224.267,70.4534


In [26]:
df_c.to_csv('temp_disaster_merge_new.csv', index = False)

In [27]:
df_c.head()

,disaster_number,STATE,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,fips,place_code,designated_area,...,month_x,month_year,TEMPERATURE_ID,DATE,Country,Fahrenheit,AverageTemperatureF,AverageTemperatureUncertaintyF,year_y,month_y
0,1796,AK,DR,2008-09-26,2008,Severe Storm(s),"Severe Storms, Flooding, Landslides, And Mudsl...",2090,99090,Fairbanks North Star (Borough),...,9,2008-09,1398,2008-09-01,United States,74.696,40.5392,32.4518,2008,9
1,1796,AK,DR,2008-09-26,2008,Severe Storm(s),"Severe Storms, Flooding, Landslides, And Mudsl...",2068,99068,Denali (Borough),...,9,2008-09,1398,2008-09-01,United States,74.696,40.5392,32.4518,2008,9
2,1796,AK,DR,2008-09-26,2008,Severe Storm(s),"Severe Storms, Flooding, Landslides, And Mudsl...",2185,99185,North Slope (Borough),...,9,2008-09,1398,2008-09-01,United States,74.696,40.5392,32.4518,2008,9
3,1796,AK,DR,2008-09-26,2008,Severe Storm(s),"Severe Storms, Flooding, Landslides, And Mudsl...",2290,86650,Yukon Koyukuk Regional Educational Attendance ...,...,9,2008-09,1398,2008-09-01,United States,74.696,40.5392,32.4518,2008,9
4,1440,AK,DR,2002-11-08,2003,Earthquake,Earthquake,2170,99170,Matanuska-Susitna (Borough),...,11,2002-11,1328,2002-11-01,United States,-27.373,20.1254,32.4266,2002,11
